good reference for stacking: https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14335
        

### Loading needed libs

In [23]:
import numpy as np
import math
import pandas as pd

import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import *

from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import *
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import pickle
from sklearn.svm import SVC
from xgboost import XGBClassifier
from time import time

### Finding the features

In [16]:
# make predictions for multinoulli NB
df = pd.read_csv('reddit_train.csv')
df2 = pd.read_csv('reddit_test.csv')
# df = df.sample(5000, random_state=1).copy()

In [17]:
df['category_id'], mapping = df['subreddits'].factorize()

In [18]:
tfidf = TfidfVectorizer()
features = tfidf.fit_transform(df.comments)
features_test = tfidf.transform(df2.comments)

labels = df.category_id

### Code for fine tuning and check which models are the best

In [88]:
for C in [0.01, 0.1, 1, 10]:
    gNB = LogisticRegression(C=C)
    gNB.fit(features[:50000,:],labels[:50000])
    print(C, np.mean(gNB.predict(features[50000:,:])==labels[50000:]))

0.01 0.3803
0.1 0.4564
1 0.53595
10 0.53895


### Extracting features from a sklearn model

In [19]:
def getFeatures(classifier):
    classifier.fit(features, labels)
    f1 = classifier.predict_proba(features)
    f2 = classifier.predict_proba(features_test)
    print(' training accuracy: ', np.mean(classifier.predict(features)==labels))
    return f1, f2

In [20]:
m1_train, m1_test = getFeatures(MultinomialNB(0.2))
m2_train, m2_test = getFeatures(LogisticRegression(C=1))

 training accuracy:  0.8171285714285714


/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


 training accuracy:  0.7188571428571429


### Extracting features from pytorch models (saved)

In [21]:
# load predictions
def pickle2np(filename="preds/bert-preds-test.p"):
    tmp = pickle.load( open( filename, "rb" ) )
    preds = list()
    for i in range(len(tmp)):
        for j in range(len(tmp[i])):
            preds.append(list(tmp[i][j]))
            
    return np.array(preds)

m3_train = pickle2np("preds/bert-preds-train.p")
m3_test = pickle2np("preds/bert-preds-test.p")
m4_train = pickle2np("preds/xlnet-preds-train.p")
m4_test = pickle2np("preds/xlnet-preds-test.p")

In [30]:
# stacking the features
features = np.concatenate((m1_train, m2_train, m3_train, m4_train),axis=1)
test_features = np.concatenate((m1_test, m2_test, m3_test, m4_test),axis=1)
labels = df.category_id

In [ ]:
# uncomment if you want to work on a smaller sample
samples = np.random.randint(70000,size=(1000))
features = features[samples,:]
labels = df.category_id[samples]
features.shape

In [24]:

models = [
#     RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
#     LogisticRegression(),
    LinearSVC(),
    XGBClassifier(),
    AdaBoostClassifier()
#     GaussianNB(),
#     KNeighborsClassifier(),
#     GradientBoostingClassifier()
]

# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
    model_name = model.__class__.__name__
    print(model_name)
    start_time = time()
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    print('time taken: ', time()-start_time)
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

LinearSVC


/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to c

time taken:  2.7757182121276855
XGBClassifier
time taken:  23.492751121520996
AdaBoostClassifier
time taken:  2.286252021789551


In [25]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
AdaBoostClassifier,0.499380,0.123815
LinearSVC,0.741195,0.040745
XGBClassifier,0.804769,0.058318


### fine tuning the meta learner

In [29]:
for eta in [0.01, 0.1, 0.2, 0.3, 1]:
    gNB = XGBClassifier(learning_rate=eta)
    gNB.fit(features[:900,:],labels[:900])
    print(eta, np.mean(gNB.predict(features[900:,:])==labels[900:]))

0.01 0.89
0.1 0.9
0.2 0.88
0.3 0.88
1 0.85


### fit classifier on full dataset

In [31]:
reg = XGBClassifier()
start_time = time()
reg.fit(features,labels)
print(time()-start_time)

709.6374371051788


In [32]:
# get predictions and training accuracy
print(np.mean(reg.predict(features)==labels))
preds = reg.predict(test_features)

0.8832428571428571


In [33]:
preds.shape

(30000,)

In [34]:
test_preds = pd.DataFrame()
test_preds['Id'] = df2['id']
test_preds['Category'] = mapping[preds]


In [35]:
test_preds.to_csv("test.csv", index=False)
from IPython.display import FileLink, FileLinks
FileLink('test.csv')

/network/home/penmetss/comp551/test.csv